In [1]:
import os

import psycopg
import pandas as pd
import numpy as np
import mlflow
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error



from dotenv import load_dotenv, find_dotenv

In [2]:
# подгружаем .env
load_dotenv()

True

In [3]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [4]:
TABLE_NAME = 'clean_prediction_price_estate'

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df.head(2)

,id,floor,is_apartment,kitchen_area,living_area,rooms,total_area,price,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,16,11,0,9.2,31.799999,2,54.0,7680000,20965,2008,4,55.501740,37.580624,2.64,156,14,1
1,18,3,0,10.0,45.000000,3,73.0,17300000,14741,1987,4,55.627918,37.512028,2.64,327,17,1


In [6]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = s3_access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = s3_secret_access_key

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

## Logging model

In [7]:
EXPERIMENT_NAME = 'logging_base_model_mmakarov'
RUN_NAME = 'stage_1'
REGISTRY_MODEL_NAME = 'base_model_mmakarov'

In [8]:
import joblib

# Загружаем модель
with open('fitted_model.pkl', 'rb') as fd:
    fit_pipeline = joblib.load(fd)

model = fit_pipeline.named_steps['model']
# Получаем наши признаки и скоры (входные и выходные данные)
transformed_data = fit_pipeline.named_steps['preprocessor'].transform(df)

/home/mle-user/mle_projects/mle-project-sprint-2-v001/.venv_project_module_2/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.1.post1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-project-sprint-2-v001/.venv_project_module_2/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.1.post1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/mle-user/mle_projects/mle-project-sprint-2-v001/.venv_project_mod

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(transformed_data, df['price'], 
                                                    test_size=0.2, 
                                                    shuffle=False)

print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")


prediction = model.predict(X_test)

# расчёт метрик качества
metrics = {}

r2 = r2_score(y_test, prediction)
mse = mean_squared_error(y_test, prediction)
mae = mean_absolute_error(y_test, prediction)
rmse = np.sqrt(mse)

# сохранение метрик в словарь
metrics["mae"] = mae
metrics["mse"] = mse
metrics["rmse"] = rmse
metrics["r2"] = r2 


# настройки для логирования в MLFlow
pip_requirements = '../../requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:

    run_id = run.info.run_id
    mlflow.log_metrics(metrics)
    mlflow.log_params(model.get_all_params())
    # cv_info = mlflow.sklearn.log_model(cv, artifact_path='cv')
    model_info = mlflow.catboost.log_model(
        await_registration_for=60, 
        cb_model=model, 
        signature=signature, 
        artifact_path='models',
        registered_model_name=REGISTRY_MODEL_NAME, 
        input_example=input_example, 
        pip_requirements=pip_requirements)



Размер выборки для обучения: (85072, 6)
Размер выборки для теста: (21269, 6)


Registered model 'base_model_mmakarov' already exists. Creating a new version of this model...
2025/04/06 10:25:32 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: base_model_mmakarov, version 2
Created version '2' of model 'base_model_mmakarov'.
